# Implementando uma máquina de estados

### Exercício 1

Vamos implementar uma máquina de estados como no diagrama abaixo. Este exercício é importante para a fixação de alguns conceitos da linguagem **Python** que serão necessários para o correto aproveitamento das demais atividades do curso.

![title](https://www.thiengo.com.br/img/post/normal/f8vu2jt9hh2fcendvulf6qb7t2528cd4c8ac16a31e05ff7bb569678aa3.png)

In [ ]:
class MaquinaEstados():

    def __init__(self, moedas=None, refrigerantes=0):
        self.moedas = moedas
        self.refrigerantes = refrigerantes
    
    def sem_moeda(self):
        if self.moedas is None or self.moedas <= 0:
            print('Nenhuma Moeda')
        if self.moedas:
            print('Inseriu Moeda')
            self.com_moeda()

    def com_moeda(self):
        print('Despejar!')
        self.venda()
    
    def venda(self):
        print('Venda realizada')
        self.moedas -= 1
        self.refrigerantes -= 1
        if self.refrigerantes > 0:
            self.sem_moeda()
        else:
            self.sem_refrigerante()
    
    def sem_refrigerante(self):
        print('Sem refrigerante')
        
    def inicio(self):
        self.sem_moeda()

In [ ]:
maq = MaquinaEstados(3, 3)

In [ ]:
maq.inicio()

## Rotina tratamento de dados

Temos que pensar que da mesma maneira que trabalhamos uma base para o desenvolvimento de qualquer tipo de modelo, também precisamos trabalhar os dados em que este modelo será responsável por responder.

Não há motivos em ter todo o trabalho de desenvolvimento de inteligência aplicada a tomada de decisão, se o trabalho não for utilizado em um ambiente produtivo.

Existem muitas formas de servir um modelo baseado em IA, uma delas é através de um serviço web. Este é padrão que vamos estudar hoje.

Imagine que nosso modelo, após o período de treinamento, é disponibilizado em um servidor web. Este servidor aguarda receber um pedido de processamento de informação.

Este pedido é realizado através de um método POST, onde um json com os dados de entrada é esperado pelo serviço na estrutura descrita abaixo:

```js
{
    "unidade" : "PERDIZES",
    "dt_agendamento" : "Dia Normal",
    "mes" : "Dezembro",
    "dia_da_semana" : "Terça",
    "periodo_do_dia" : "Manha",
    "fonte_pagadora" : "Convenio",
    "sexo" : "Masculino",
    "exame" : "Teste Ergometrico",
    "area" : "METODOS GRAFICOS",
    "idade" : "47",
    "bairro" : "TAMBORE",
    "cidade" : "SANTANA DE PARNAIBA",
    "estado" : "SP"
}
```

O cliente espera que o serviço responda a previsão do modelo com os dados fornecidos, essa resposta será enviada da mesma maneira seguindo a estrutura abaixo:

```js
{
    "model_response" : 0
}
```

Se o cliente souber como interpretar a resposta, temos então uma arquitetura de request e response definidos.

É importante que no momento do treinamento deste modelo, nós tenhamos o cuidado de pensar em como vamos disponibilizar essas informações e também como vamos processar os dados que recebemos.

O objetivo desta atividade é demonstrar alguns cuidados que devemos ter com o preparo da base pensando nesses pontos.

#### Lendo um arquivo Excel

Para modelar o problema recebemos da área de negócios um arquivo em excel com os dados de agendamento de exames.

In [ ]:
data = pd.read_excel('https://github.com/pgiaeinstein/otmz-mlp/raw/master/saida_aula.xlsx')
data.head()

Note que o nome das colunas é diferente do padrão de estrutura que adotamos acima, vamos corrigir isso para não cometer nenhum erro no futuro:

In [ ]:
colunas = ['unidade', 'dt_agendamento', 'mes', 'dia_da_semana', 'periodo_do_dia', 'fonte_pagadora', 'sexo', 'exame', 'area', 'idade', 'bairro', 'cidade', 'estado', 'status']

data.columns = colunas

In [ ]:
data.head()

Para padronizar os dados, vamos modificar o conteúdo das colunas para que todas as letras sejam minúsculas e vamos também remover qualquer acento das palavras.

Repare que em uma das colunas temos valores numéricos, podemos ter um resumo do nosso dataset utilizando o comando abaixo:

In [ ]:
data.info()

Repare que todos os valores textuais em nossa base tem o tipo `object`. Já a coluna *idade* possui tipo `int64`.

Essa coluna não será modificada, temos que descobrir uma forma de identificar o tipo de cada coluna individualmente, isso pode ser feito como demonstrado abaixo:

In [ ]:
for coluna in data.columns:
    print(f'Coluna "{coluna}" tem tipo "{data[coluna].dtype}"')

Agora que temos como diferenciar os tipos por colunas, podemos padronizar a informação.

Para remover os acentos das palavras utilizaremos uma biblioteca do python chamada `unidecode`. 

In [ ]:
!pip install unidecode
from unidecode import unidecode

In [ ]:
for coluna in data.columns:
    if data[coluna].dtype == 'object':
        data[coluna] = data[coluna].str.lower()
        data[coluna] = data[coluna].apply(unidecode)

In [ ]:
data.head()

Temos agora que criar uma forma de representar essa informação para que seja possível de um algoritmo interpretar. Como temos um problema dito binário, ou seja, há duas possibilidades de resposta:

0. nao veio
1. veio

Vamos modificar a coluna `status` para respeitar este novo formato.

In [ ]:
status_dict = {
    'veio' : 1,
    'nao veio' : 0
}

data['status'] = data['status'].map(status_dict)

In [ ]:
data.head(10)

Reparem que a ordem pouco importa, o que fizemos foi criar um dicionário que vai servir como nosso tradutor de informação. Vamos fazer a mesma coisa para todas as colunas.

Nada nos impede de criar esses dicionários na mão como no exemplo acima, porém, quando temos um número muito grande de classes, este trabalho pode ser complexo e podemos gerar algum erro.

A biblioteca ScikitLearn têm algumas classes prontas para realizar este tipo de processamento. Iremos utilizar uma dessas classes para nos auxiliar nesta tarefa.

Agora que temos os conteúdos padronizados, o ideal é separar o que temos como dados de `input` (features) e `output` (meta ou target).

In [ ]:
features = data.iloc[:, :-1]
meta     = data.iloc[:, 13]

In [ ]:
features.head(10)

In [ ]:
meta.head(10)

Agora que temos o dataset devidamente separado, podemos criar nossos dicionários. Como dito acima, utilizaremos a classe [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) para a tarefa:

In [ ]:
from sklearn.preprocessing import LabelEncoder

cols   = features.dtypes==object
labels = features.columns[cols].tolist()

dicionarios = dict()

for label in labels:
    le = LabelEncoder()
    le.fit(features[label])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    dicionarios[label] = le_name_mapping

O resultado da operação é um grande dicionário de dicionários, veja abaixo:

In [ ]:
dicionarios

O problema é que nossos índices iniciam em 0, por padrão, vamos utilizar o 0 em caso de dado não identificado. Para cada chave iremos somar o valor de 1, assim resolvemos esse problema do 0.

In [ ]:
for label in labels:
    dict_tmp = dicionarios[label]

    for key, value in dict_tmp.items():
        dict_tmp[key] = value + 1

In [ ]:
dicionarios

Precisamos agora de uma maneira de salvar esses dicionários para uso posterior. Imagine que vamos traduzir informação nova para classificação, não necessariamente temos que treinar o modelo toda vez que uma nova informação é disponibilizada.

Poderíamos salvar esses dicionários em arquivos json para facilitar a interoperabilidade entre sistemas, mas vamos salvar em objetos Python através da biblioteca `pickle`. Veja o exemplo:

In [ ]:
import pickle

In [ ]:
unidade_dict = dicionarios['unidade']

In [ ]:
unidade_dict

In [ ]:
pickle.dump(unidade_dict ,open('unidade_dict.pk', 'wb'))

Através do método `dump` podemos salvar o dicionário para uso posterior. Para carregar novamente este dicionário, utilizamos o método `load`:

In [ ]:
load_unidade_dict = pickle.load(open('unidade_dict.pk', 'rb'))

In [ ]:
unidade_dict

In [ ]:
load_unidade_dict

Podemos automatizar a criação destes arquivos da seguinte forma:

In [ ]:
for label in labels:
    dict_tmp = dicionarios[label]
    pickle.dump(dict_tmp ,open(f'__dict_{label}.pk', 'wb'))

O que fizemos foi criar um arquivo por dicionário seguindo o padrão "**dict_nome_da_coluna.pk**".

Podemos também modificar todo o conteúdo do nosso dataset seguindo esse padrão representativo numérico que criamos:

In [ ]:
for label in labels:
    features[label] = features[label].map(dicionarios[label])

In [ ]:
features.head()

## Problema

Crie uma classe para automatizar a tradução dos dados novos que você receberá neste projeto. Utilize dos arquivos gerados para criar uma forma de automatizar o processo.

Para testar sua classe, utilize a estrutura abaixo:

```js
{
    "unidade" : "PERDIZES",
    "dt_agendamento" : "Dia Normal",
    "mes" : "Dezembro",
    "dia_da_semana" : "Terça",
    "periodo_do_dia" : "Manha",
    "fonte_pagadora" : "Convenio",
    "sexo" : "Masculino",
    "exame" : "Teste Ergometrico",
    "area" : "METODOS GRAFICOS",
    "idade" : "47",
    "bairro" : "TAMBORE",
    "cidade" : "SANTANA DE PARNAIBA",
    "estado" : "SP"
}
```

O resultado deverá ser um array dos valores inteiros relativos as classes criadas no exemplo acima. A ordem dos valores deve respeitar a mesma ordem do dicionário dados.

> Dica:
> Tente rodar o seguinte comando: `dados.get('unidade', None)` verifique o valor obtido como output

In [ ]:
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_area.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_bairro.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_cidade.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_dia_da_semana.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_dt_agendamento.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_estado.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_exame.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_fonte_pagadora.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_mes.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_periodo_do_dia.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_sexo.pk
!wget https://github.com/pgiaeinstein/otmz-mlp/raw/master/dict_data/dict_unidade.pk

In [1]:
dados = {
    "unidade" : "PERDIZES",
    "dt_agendamento" : "Dia Normal",
    "mes" : "Dezembro",
    "dia_da_semana" : "Terça",
    "periodo_do_dia" : "Manha",
    "fonte_pagadora" : "Convenio",
    "sexo" : "Masculino",
    "exame" : "Teste Ergometrico",
    "area" : "METODOS GRAFICOS",
    "idade" : "47",
    "bairro" : "TAMBORE",
    "cidade" : "SANTANA DE PARNAIBA",
    "estado" : "SP"
}